## Install Dependencies

In [ ]:
!git clone https://github.com/GFDRR/caribbean-rooftop-classification.git
!pip -q install -U segment-geospatial
!pip -q install -U git+https://github.com/IDEA-Research/GroundingDINO.git

In [ ]:
!pip -q install leafmap localtileserver huggingface_hub

In [ ]:
%cd caribbean-rooftop-classification/
!git checkout add/tutorials
%cd tutorials

## Imports and Setup

In [ ]:
import os
import gdown

import matplotlib.pyplot as plt
import rasterio as rio
import geopandas as gpd
import leafmap

from rasterio.plot import show
from shapely.geometry import box
from samgeo import SamGeo

import sys
sys.path.insert(0, "../")
from utils import langsam_utils

%load_ext autoreload
%autoreload 2

## Download Image

In [ ]:
image_dir = "../data/rasters/drone/"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

filename = "drone.tif"
image_file = os.path.join(image_dir, filename)

gid = "1SBqJuGfCmrL9YXbwdoK3uOkbPSE4uFGJ"
if not os.path.isfile(image_file):
    url = f"https://drive.google.com/uc?id={gid}"
    gdown.download(url, image_file, quiet=False)

## Load Image

In [ ]:
image = rio.open(image_file)
data = gpd.GeoDataFrame({"id":1,"geometry":[box(*image.bounds)]}, crs=image.crs)
data = data.to_crs("EPSG:4326")
bbox = data.geometry.values[0]
centroid = bbox.centroid

map = leafmap.Map(center=[centroid.y, centroid.x], zoom=20)
map.add_raster(image_file, layer_name="Image", zoom=20)
map

## LangSAM

In [ ]:
text_prompt = "house"
model = langsam_utils.LangSAM()

In [ ]:
tiles = langsam_utils.generate_tiles(image_file, size=3000)
print('Data dimensions: {}'.format(tiles.shape))
tiles.head(3)

In [ ]:
image = rio.open(image_file)
fig, ax = plt.subplots(figsize=(15,15))
tiles.plot(facecolor="none", edgecolor='red', ax=ax, aspect=1)
show(image, ax=ax);

In [ ]:
langsam_utils.show_crop(image_file, [tiles.iloc[20]['geometry']])

In [ ]:
out_dir = './temp/'
out_file = 'merged.gpkg'

for index in tqdm(range(len(tiles)), total=len(tiles)):
    shape = [tiles.iloc[index]['geometry']]
    langsam_utils.predict_crop(
        image_file, shape, model, out_dir, index
    )
polygons = langsam_utils.merge_polygons(out_dir)
langsam_utils.connect_polygons(polygons, merged_file=out_file, intersect_ratio=0.6)

In [ ]:
map = leafmap.Map(center=[centroid.y, centroid.x], zoom=20)
map.add_raster(image_file, layer_name="Image", zoom=20)
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
map.add_vector(out_file, layer_name="Vector", style=style)
map